<a href="https://colab.research.google.com/github/WangKehanK/CS505/blob/main/Twitter_User_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import json
import os

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
df = pd.read_csv("/content/drive/MyDrive/Twitter_user_handles_to_predict.csv", encoding = "utf-8")
df_labeled = pd.read_csv("/content/drive/My Drive/Twitter_users_labeled_with_age_and_gender.csv", encoding = "latin-1")

In [36]:
# Get authentication for tweepy from auth.txt
# auth.txt was uploaded before in order to use m3inference
keys = ''
with open(f'/content/drive/MyDrive/auth.txt', 'r') as reader:
  keys=reader.read()
keys = keys.split('\n')
# for i in range(len(keys)):
#   keys[i] = keys[i].split('"')[1];


In [38]:
# Setup tweepy
import tweepy
api_key = keys[0]
api_secret = keys[1]
access_token = keys[2]
access_secret = keys[3]
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [39]:
# Try first user
user = df['Username']
userID = user[0]
print(userID)
tweets = api.user_timeline(screen_name=userID,count=100,include_rts = False,tweet_mode = 'extended')

JayHolz410


In [40]:
# Extract full text from tweets
tweet_text = [tweet._json['full_text'] for tweet in tweets]
print(len(tweet_text))
tweet_text[0]

91


'Yeah that’s enough Twitter for tonight ✌🏼'

In [43]:
# Extract latest 100 tweets using api.user_timeline for each user
import time

all_tweets = []
for userID in user:
  try:
    tweets = api.user_timeline(screen_name=userID, 
                              count=100,
                              include_rts = False,
                              tweet_mode = 'extended')
    all_tweets.extend(tweets)
    print('Number of tweets extracted {}'.format(len(all_tweets)))

  except tweepy.error.TweepError as t:
        if t.message[0]['code'] == 50: # The code corresponding to the user not found error
            print("screen_name that failed=",  userID)
            user.remove(userID)

        elif t.message[0]['code'] == 88: # The code for the rate limit error
            time.sleep(15) # Sleep for 15 seconds


# for testing purpose, print each user's 3 most recent tweets
  # for info in tweets[:3]:
  #    print("ID: {}".format(info.id))
  #    print(info.created_at)
  #    print(info.full_text)
  #    print("\n")

Number of tweets extracted 91
Number of tweets extracted 159
Number of tweets extracted 244
Number of tweets extracted 290
Number of tweets extracted 340
Number of tweets extracted 406
Number of tweets extracted 474
Number of tweets extracted 538
Number of tweets extracted 581
Number of tweets extracted 610


AttributeError: ignored

In [ ]:
outtweets = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx,tweet in enumerate(all_tweets)]

df = DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])
df.to_csv('%all_tweets.csv' % userID,index=False)
df.head(3)


TODO: Using BERT to extract fixed feature vectors

https://github.com/google-research/bert#using-bert-to-extract-fixed-feature-vectors-like-elmo

In [ ]:
age_no_na = df_labeled[df_labeled['human.labeled.age'].notna()]
age_no_na